# Risk-Averse Reward Model Training

This notebook implements the complete risk aversion experiment for training **Qwen3-8B** to prefer risk-averse choices over risk-neutral ones.

## Features
- **CSV Data Loading**: Loads scenarios from `11_7_low_stakes_training_set.csv`
- **CARA vs LINEAR Utility**: Trains on CARA (risk-averse) vs LINEAR (risk-neutral) best options


**Memory Optimized for 8B Model:** Uses gradient checkpointing, smaller batch size, and fp16 for memory efficiency


## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers datasets accelerate torch pandas numpy scikit-learn matplotlib seaborn

print("✓ Dependencies installed successfully!")

## 2. Import Libraries

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
import json
from typing import List, Dict, Tuple
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 3. Data Loading Class

Loads risk scenarios from CSV file with proper prompt modification and grouping by situation_id.

In [ ]:
class RiskAversionDataLoader:
    """Load and process data from CSV file for risk aversion training"""
    
    def __init__(self, csv_file_path="11_7_low_stakes_training_set.csv"):
        self.csv_file_path = csv_file_path
        
    def load_and_process_data(self) -> pd.DataFrame:
        """Load CSV data and process it for training
        
        New format (11_7_low_stakes_training_set.csv):
        - Multiple rows per situation (one per option)
        - is_best_cara = True marks risk-averse option
        - is_best_linear = True marks risk-neutral option
        - option_index is 0-indexed, prompts use 1-indexed numbers
        """
        # Check if CSV file exists
        if not os.path.exists(self.csv_file_path):
            raise FileNotFoundError(
                f"Required data file '{self.csv_file_path}' not found. "
                f"Please ensure the CSV file is uploaded to Colab."
            )
        
        # Load the CSV file
        df = pd.read_csv(self.csv_file_path)
        print(f"Loaded {len(df)} rows from {self.csv_file_path}")
        
        # Check required columns exist (new format)
        required_columns = ['situation_id', 'prompt_text', 'option_index', 'is_best_cara', 'is_best_linear']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(
                f"Missing required columns in CSV: {missing_columns}. "
                f"Available columns: {list(df.columns)}"
            )
        
        # Group by situation_id to get unique situations
        situations = []
        situations_skipped = 0
        
        for situation_id, group in df.groupby('situation_id'):
            # Find risk-averse option (CARA best)
            cara_rows = group[group['is_best_cara'] == True]
            if len(cara_rows) == 0:
                situations_skipped += 1
                continue
            cara_option = cara_rows.iloc[0]
            
            # Find risk-neutral option (LINEAR best)
            linear_rows = group[group['is_best_linear'] == True]
            if len(linear_rows) == 0:
                situations_skipped += 1
                continue
            linear_option = linear_rows.iloc[0]
            
            # Get prompt text from first row (same for all options)
            prompt_text = group.iloc[0]['prompt_text']
            
            # Convert 0-indexed option_index to 1-indexed option numbers (as shown in prompt)
            correct_label = str(cara_option['option_index'] + 1)
            incorrect_label = str(linear_option['option_index'] + 1)
            
            situations.append({
                'situation_id': situation_id,
                'prompt_text': prompt_text,
                'correct_label': correct_label,  # Risk-averse option number
                'incorrect_label': incorrect_label,  # Risk-neutral option number
                'num_options': len(group)
            })
        
        if situations_skipped > 0:
            print(f"Warning: Skipped {situations_skipped} situations missing CARA or LINEAR best options")
        
        result_df = pd.DataFrame(situations)
        print(f"Processed into {len(result_df)} unique situations")
        
        # Display sample data
        if len(result_df) > 0:
            sample = result_df.iloc[0]
            print(f"\nSample situation:")
            print(f"Prompt: {sample['prompt_text'][:200]}...")
            print(f"Risk-averse choice (CARA best): Option {sample['correct_label']}")
            print(f"Risk-neutral choice (LINEAR best): Option {sample['incorrect_label']}")
            print(f"Number of options in this situation: {sample['num_options']}")
        
        return result_df

print("✓ RiskAversionDataLoader defined")